# Bonus: Temperature Analysis I

In [1]:
import pandas as pd
from datetime import datetime as dt

In [2]:
# "tobs" is "temperature observations"
df = pd.read_csv('Resources/hawaii_measurements.csv')
df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,NaN,73


In [3]:
# Convert the date column format from string to datetime
df['date'] = pd.to_datetime(df['date'])
df.dtypes

station            object
date       datetime64[ns]
prcp              float64
tobs                int64
dtype: object

In [4]:
# Set the date column as the DataFrame index
df = df.set_index('date', drop = True)

In [7]:
# Drop the date column
df
# HUH??? How/why would we drop a column once it is the index? The column doesn't exist...

,station,prcp,tobs
date,,,
2010-01-01,USC00519397,0.08,65
2010-01-02,USC00519397,0.00,63
2010-01-03,USC00519397,0.00,74
2010-01-04,USC00519397,0.00,76
2010-01-06,USC00519397,NaN,73
...,...,...,...
2017-08-19,USC00516128,0.09,71
2017-08-20,USC00516128,NaN,78
2017-08-21,USC00516128,0.56,76


### Compare June and December data across all years 

In [8]:
from scipy import stats

In [9]:
# Filter data for desired months
des_df = df.loc[(df.index.month == 12) | (df.index.month == 6)]
des_df

,station,prcp,tobs
date,,,
2010-06-01,USC00519397,0.00,78
2010-06-02,USC00519397,0.01,76
2010-06-03,USC00519397,0.00,78
2010-06-04,USC00519397,0.00,76
2010-06-05,USC00519397,0.00,77
...,...,...,...
2017-06-26,USC00516128,0.02,79
2017-06-27,USC00516128,0.10,74
2017-06-28,USC00516128,0.02,74


In [10]:
# Identify the average temperature for June
avg_by_month = des_df.groupby(by = (des_df.index.month)).mean()
june = avg_by_month.loc[6, 'tobs']
print(f'The average temperature for June is {round(june,2)}.')

The average temperature for June is 74.94.


In [11]:
# Identify the average temperature for December
dec = avg_by_month.loc[12, 'tobs']
print(f'The average temperature for June is {round(dec,2)}.')

The average temperature for June is 71.04.


In [23]:
# Create collections of temperature data
june_series = des_df.loc[des_df.index.month == 6, 'tobs']
dec_series = des_df.loc[des_df.index.month == 12, 'tobs']
# After conversation with classmate Mir Ahmed, decided to also create collections of means by station in each month. 
## If we have 9 station means in each month, we can view these collections as paired for t-test.
june_by_station = df.loc[(df.index.month == 6)].groupby('station')['tobs'].mean()
dec_by_station = df.loc[(df.index.month == 12)].groupby('station')['tobs'].mean()

In [27]:
# Using all measurements needs to be unpaired...the data sets are not the same size
stats.ttest_ind(june_series,dec_series)

Ttest_indResult(statistic=31.60372399000329, pvalue=3.9025129038616655e-191)

In [25]:
# Run paired t-test
stats.ttest_rel(june_by_station,dec_by_station)

Ttest_relResult(statistic=6.95696617044294, pvalue=0.00011759380231523222)

### Analysis

After running an unpaired t-test (unpaired as the groups have a 'commmon thread' but are not the same group), the returned p-value was approx. 3.90e-191 (i.e., very close to zero). This indicates that the null hypothesis that average (mean) temperatures of Hawaii in June and December are statistically similar is true.

After having a conversation with a classmate as mentioned above, I decided to look at the mean temperture of each station in each month. As we can pair this data (both sets are the means at the same 9 stations), I ran a paired t-test. The result was still very close to zero. This was a good exercise, as I can see that the p-value was larger and the statistic smaller for the paired test, as the data sets compared were much smaller when using means.